In [1]:
import torch
import os
from tqdm import tqdm, trange
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from vgg_dataset import *
from models import *
from train_utls import *

In [2]:
root = "/mnt/new_volume2/vgg_sound_emb"
partition = "train"
data_dir = f"{root}/{partition}"
print(f"Loading {partition} data from {data_dir}")

Loading train data from /mnt/new_volume2/vgg_sound_emb/train


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# vgg_sound = LargeVideoDataset(data_dir, subset_ratio = 0.2)
vgg_sound = InMemoryVideoDataset(data_dir, subset_ratio=0.8)

Loading 146100 samples into memory as torch.float16 …


Caching data:  53%|█████▎    | 77163/146100 [10:28<09:19, 123.18it/s]

# DataLoader

In [5]:
val_ratio = 0.1
test_ratio = 0.1

total_len = len(vgg_sound)
val_len = int(total_len * val_ratio)
test_len = int(total_len * test_ratio)
train_len = total_len - val_len - test_len
train_dataset, val_dataset, test_dataset = random_split(
    vgg_sound, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42), 
)


In [6]:
batch_size = 2048
num_workers = 8

In [7]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=num_workers,
    pin_memory=True
)

In [8]:
clip_feat, clap_feat = next(iter(train_loader))
print("Clip:", clip_feat.device, clip_feat.dtype)
print("Clap:", clap_feat.device, clap_feat.dtype)

Clip: cpu torch.float16
Clap: cpu torch.float16


# Wandb

In [9]:
import wandb

# Model

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CosineLoss(nn.Module):
    def __init__(self, margin=0.8):  # 建议 margin 设得稍高一点
        super(CosineLoss, self).__init__()
        self.margin = margin

    def forward(self, output, target):
        # output: (B, 512)
        # target: (B, 1, 512) or (B, 512)
        if target.ndim == 3:
            target = target.squeeze(1)
        cos_sim = F.cosine_similarity(output, target, dim=1)  # (B,)
        loss = torch.mean(torch.clamp(self.margin - cos_sim, min=0))
        return loss


In [11]:
print(device)

cuda


In [ ]:
# (input_dim=512, hidden_dim=1024, output_dim=512, lstm_hidden=256, lstm_layers=1)
configs = {"input_dim": 512, "hidden_dim": 1024, "output_dim": 512, "lstm_hidden": 256, "lstm_layers": 1 ,"dropout": 0.3}

In [ ]:
# model = V2AMapperMLPImproved(input_dim=512, hidden_dim=2048, output_dim=512).to(device)
model = V2AMapperBiLSTM(input_dim=512, hidden_dim=1024, output_dim=512, lstm_hidden=256, lstm_layers=1,dropout=0.3).to(device)

In [13]:
# show me the mode summary 
from torchsummary import summary
# model = V2AMapperMLP(input_dim=512, hidden_dim=512, output_dim=512).to(device)
# model = V2AMapperMLPImproved().to(device)

# # 改用 torchinfo 兼容性更好！
# from torchinfo import summary as torchinfo_summary
# torchinfo_summary(model, input_size=(1, 64, 512))

In [13]:
# Use wandb? Resume Training?
USE_WANDB = True

RESUME_LOGGING = False # Set this to true if you are resuming training from a previous run

# Create your wandb run

run_name = 'lstm-vggsound-simplefied_mapper' # Give your run a name, this will be used to identify the run in wandb

# If you are resuming an old run
if USE_WANDB:

    wandb.login(key="8475199febe13b3465c7d5e4a595bba7422c14fc") #TODO

    if RESUME_LOGGING:
        run = wandb.init(
            id     = "", ### Insert specific run id here if you want to resume a previous run
            resume = "must", ### You need this to resume previous runs
            project = "v2amapper", ### Project should be created in your wandb
            settings = wandb.Settings(_service_wait=300)
        )


    else:
        run = wandb.init(
            name    = run_name, ### Wandb creates random run names if you skip this field, we recommend you give useful names
            reinit  = True, ### Allows reinitalizing runs when you re-run this cell
            project = "v2amapper", ### Project should be created in your wandb account
        )

        ### Save your model architecture as a string with str(model)
        model_arch  = str(model)
        ### Save it in a txt file
        arch_file   = open("model_arch.txt", "w")
        file_write  = arch_file.write(model_arch)
        arch_file.close()

        ### log it in your wandb run with wandb.save()
        wandb.save('model_arch.txt')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ubuntu/.netrc
wandb: Currently logged in as: zianp (zianp-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [14]:
lr = 0.001
epochs = 120
# model = V2AMapperMLP(input_dim=512, hidden_dim=1024, output_dim=512).to(device)
criterion = nn.MSELoss()
# criterion = CosineLoss(margin= 1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# scheduler = torch.optim.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.00001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0.000005)
scaler = torch.amp.GradScaler(enabled=True)

ckpt_dir = "ckpts/"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

train(model, train_loader, val_loader, criterion, optimizer,scaler, scheduler, ckpt_dir = "ckpts/", num_epochs=epochs)

Epoch 1/120


Train:   0%|          | 0/15 [00:00<?, ?it/s]

[Debug] clip_feat device = cuda:0, clap_feat device = cuda:0


Train Loss: 0.9445, Val Loss: 0.9136
Model saved!
Epoch 2/120


Train:  13%|█▎        | 2/15 [00:01<00:07,  1.66it/s, loss=0.9120, lr=0.001000]

...

[Debug] clip_feat device = cuda:0, clap_feat device = cuda:0


Train Loss: 0.0015, Val Loss: 0.0018
Model saved!
Epoch 120/120


Train:  13%|█▎        | 2/15 [00:01<00:07,  1.63it/s, loss=0.0015, lr=0.000005]

[Debug] clip_feat device = cuda:0, clap_feat device = cuda:0


Train Loss: 0.0015, Val Loss: 0.0018
Model saved!
Training complete!
Best validation loss: 0.0018


In [15]:
from infer import *

/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Load AudioLDM: %s audioldm-s-full-v2
DiffusionWrapper has 185.04 M params.


/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias'

KeyboardInterrupt: 